In [3]:
from numpy.random import normal
from random import choice
from random import randrange, random
import shutil as s
import os
import pandas as pd

class Note:
    
    def __init__(self, start = 0, octave = 0):
        self.scale = ['c','d','e','f','g','a','b']
        self.current = start
        self.octave = octave
        
    def to_string(self):
        note = self.scale[self.current]
        octave_mark = ''
        if self.octave > 0:
            octave_mark = "'" * self.octave
        if self.octave < 0:
            octave_mark = "," * abs(self.octave)
        return str(self.scale[self.current]+octave_mark)
    
    def __add__(self, other):
        return self.jump(other)
        
    def __sub__(self, other):
        return self.jump(-1*other)
        
    def jump(self, steps):
        self.current += steps
        self.octave += self.current//7
        self.current %= 7
        return Note(self.current, self.octave)
        
    def relative(self, relative_to):
        return self.current + relative_to * 7
    
    def absolute(self):
        return self.current+self.octave*7
        
class Instrument:
    def __init__(self, bot = -3, top = 30, name = 'violin'):
        self.variance = 2
        self.bottom_border = bot
        self.up_border = top
        self.start_octave = 1
        self.current = Note(octave = self.start_octave)
        self.jumps = []
        self.notes = []
        self.pages = 0
        
    def gen_next(self, offset = False):
        if not offset:
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        while offset == 0: # take random offset until value is different from 0 - we don't want multiple notes one after another with the same pitch
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        # print(f"{self.current.to_string()}: {self.current.absolute()}, {offset}")
        
        while self.current.absolute() + offset < self.bottom_border: #tu jest coś nie tak bo nie ogranicza ani w górę ani w dół
            #print("{} is too low, jumping by {}".format(offset, offset+6))
            offset += 9
        while self.current.absolute() + offset > self.up_border:
            #print("{} is too high, jumping by {}".format(offset, offset-6))
            offset -= 9
            
        return self.current + offset, offset
    
    def _gen_one(self, offset = False):
        if not offset:
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        while offset == 0: # take random offset until value is different from 0 - we don't want multiple notes one after another with the same pitch
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        print(f"{self.current.to_string()}: {self.current.absolute()}, {offset}")
        
        while self.current.absolute() + offset < self.bottom_border: #tu jest coś nie tak bo nie ogranicza ani w górę ani w dół
            #print("{} is too low, jumping by {}".format(offset, offset+6))
            offset += 6
        while self.current.absolute() + offset > self.up_border:
            #print("{} is too high, jumping by {}".format(offset, offset-6))
            offset -= 6
    
        self.current = self.current + offset
        
        self.jumps.append(offset)
        self.notes.append(self.current)
    
    def gen_n(self, n):
        for i in range(n):
            if i>0:
                if i%4==0:
                    self.notes.append("|")
                if i%32==0:
                    self.notes.append("\pageBreak")
                    self.pages += 1
            self.notes.append(self.current)
            self.current = self.gen_next()[0]
        self.notes.append("|")
        self.notes.append("\pageBreak")
        self.pages += 1
                
    def to_file(self, name = 'test.ly'):
        strings = [x.to_string() if isinstance(x, Note) else x for x in self.notes]
        
        file = open(name,'w')
        file.write("""
#(set! paper-alist
  (cons '("my size" . (cons (* 21 cm) (* 2 cm))) paper-alist))

\paper {
  #(set-paper-size "my size")
  print-page-number=false
}
        
\layout {
    indent = 0\cm
    \context {
        \Score
        \omit BarNumber
    }
}
{ \\fixed c'{\hide TimeSignature\n""")
        file.write(' '.join(strings))
        file.write("\n}}")
        file.close()
        return strings
        
    def make_images(self, name='test.ly'):
        strings = self.to_file(name)
        !"C:\Program Files (x86)\LilyPond\usr\bin\lilypond.exe" -fpng test.ly
        return strings
        
    def make_csv(self, name="test.csv"):
        x = [[] for a in range(self.pages)]
        counter = 0
        for i, j in enumerate(self.notes):
            if counter==self.pages:
                break
            if j == "\pageBreak":
                counter += 1
            if j != "\pageBreak":
                if isinstance(j, Note):
                    x[counter].append(j.to_string())
                else:
                    x[counter].append(j)
                
        data = pd.DataFrame(x)
        data["path"] = [f"{i+1}.png" for i in range(data.shape[0])]
        data.to_csv(name)
        
        return data
        
    def generate_dataset_and_labels_file(self, direc="test"):
        """
        it was a long day.
        """
        self.make_csv()
        
        for file in os.listdir():
            if file.endswith(".png"):
                #pages += 1
                s.move(file, "test")
                old = os.path.join("test", file)
                new = os.path.join("test", file.replace("test-page", ""))
                os.rename(old, new)
            if file.endswith(".csv"):
                s.move(file, "test")

        
class Partiture:
    """
    This class will take care of generating intruments with specific ranges 
    and connecting them to make a page of sheet music.
    For now it's uselesss and just a placeholder for the future.
    """
    pass

def empty_test_folder():
    for file in os.listdir("test"):
        if not file.startswith("."):
            os.remove(f"test\\{file}")

In [4]:
a = Instrument()

In [5]:
empty_test_folder()

In [6]:
a.gen_n(4000)
strings = a.make_images()

Processing `test.ly'
Parsing...
test.ly:1: warning: no \version statement found, please add

\version "2.22.1"

for future compatibility
Interpreting music...[8][16][24][32][40][48][56][64][72][80][88][96][104][112][120][128][136][144][152][160][168][176][184][192][200][208][216][224][232][240][248][256][264][272][280][288][296][304][312][320][328][336][344][352][360][368][376][384][392][400][408][416][424][432][440][448][456][464][472][480][488][496][504][512][520][528][536][544][552][560][568][576][584][592][600][608][616][624][632][640][648][656][664][672][680][688][696][704][712][720][728][736][744][752][760][768][776][784][792][800][808][816][824][832][840][848][856][864][872][880][888][896][904][912][920][928][936][944][952][960][968][976][984][992][1000]
Preprocessing graphical objects...
Finding the ideal number of pages...
Fitting music on 124 or 125 pages...
Drawing systems...
Converting to PNG...
Success: compilation successfully completed


In [7]:
a.generate_dataset_and_labels_file("train")